In [1]:
from DOS import *

In [2]:
r = 0.05  #interest rate
dividend = 0.1  #divident rate
seed_train = 1 # seed for training data
seed_test = 2 # seed for reproducibility
learning_rate = 0.001 #learning rate
k = 100.0  #strike price
test_paths = 100000 # number of paths for testing

In [3]:
# Parameters
d = 5   #dimension of GBM
mu = (r - dividend) * np.ones(shape = (d, )) # drift
sigma = 0.2 * np.ones(shape = (d, ))
S0 = 100.0 * np.ones(shape = (d, )) #initial price
# we use lessp paths due to limited computational resources
base_steps = 1500 #number of training steps regardless of the dimension
total_paths = batch_size * (base_steps + d) #total number of paths, base_steps+d is the training steps, where each step we need 8192 paths
number_of_training_steps = int(total_paths / batch_size) #number of training steps

#Simulating GBM paths
X = GBM(d, mu, sigma, S0, T, dt, total_paths, seed=seed_train) #simulating GBM paths
X = X.reshape(base_steps+d, batch_size, d, 10) #reshaping to (3000+d, 8192, d, 10) to simulate 3000+d many paths of 8192 samples of 5 dimensional GBM

### A More Complex Neural Network Setting

$$
\begin{align*}
& \text{Let } X \in \mathbb{R}^d \text{ denote the input to the neural network.} \\
& \text{The neural network architecture is defined as follows:} \\
& \text{First hidden layer:} \\
& \quad H_1 = \text{ReLU}(XW_1 + b_1) \\
& \quad \text{where } W_1 \in \mathbb{R}^{d \times (d+50)} \text{ and } b_1 \in \mathbb{R}^{d+50} \text{ are the weight matrix and bias vector of the first linear transformation, respectively.} \\
& \quad H_1' = \text{BatchNorm1d}(H_1) \\
& \quad H_1'' = \text{Dropout}(H_1') \\
& \text{Second hidden layer:} \\
& \quad H_2 = \text{ReLU}(H_1''W_2 + b_2) \\
& \quad \text{where } W_2 \in \mathbb{R}^{(d+50) \times (d+50)} \text{ and } b_2 \in \mathbb{R}^{d+50} \text{ are the weight matrix and bias vector of the second linear transformation, respectively.} \\
& \quad H_2' = \text{BatchNorm1d}(H_2) \\
& \quad H_2'' = \text{Dropout}(H_2') \\
& \text{Third hidden layer:} \\
& \quad H_3 = \text{ReLU}(H_2''W_3 + b_3) \\
& \quad \text{where } W_3 \in \mathbb{R}^{(d+50) \times (d+50)} \text{ and } b_3 \in \mathbb{R}^{d+50} \text{ are the weight matrix and bias vector of the third linear transformation, respectively.} \\
& \quad H_3' = \text{BatchNorm1d}(H_3) \\
& \quad H_3'' = \text{Dropout}(H_3') \\
& \text{Output layer:} \\
& \quad \hat{y} = \text{Sigmoid}(H_3''W_4 + b_4) \\
& \quad \text{where } W_4 \in \mathbb{R}^{(d+50) \times 1} \text{ and } b_4 \in \mathbb{R} \text{ are the weight matrix and bias vector of the output layer, respectively.} \\
& \text{The model parameters are randomly initialized using a normal distribution with mean 0 and standard deviation 1.}
\end{align*}
$$
The advanced randomized neural network offers greater complexity and regularization compared to the naive model, with 3 layers instead of 2 layers in the original model, which enhances its capability to capture intricate patterns in data. This is particularly advantageous in deep optimal stopping scenarios where complex decision-making processes are involved. However, its increased complexity may lead to higher computational costs and potential overfitting issues.

In [4]:
# training the model for each stopping time
for i in range(N-1, 0, -1):
    model = create_randomized_model(d).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    train(X, r, k, dt, model, i, optimizer, number_of_training_steps, batch_size)
    # store the model and the stopping decision function and stopping time function
    F[i] = model
    f[i] = lambda x, i=i: fi(x, i, F) #  store the stopping decision function
    l[i] = lambda x, i=i: li(x, i, f, l) # store the stopping time function

the model for 8th stopping time has been trained
the model for 7th stopping time has been trained
the model for 6th stopping time has been trained
the model for 5th stopping time has been trained
the model for 4th stopping time has been trained
the model for 3th stopping time has been trained
the model for 2th stopping time has been trained
the model for 1th stopping time has been trained


In [5]:
# test the model on another set of paths
X = GBM(d, mu, sigma, S0, T, dt, test_paths, seed = seed_test)
g_val = g(X, r, k, dt) # g values at stopping times
X = torch.from_numpy(X).float().to(device) # convert X to a tensor
Z = g_val[range(test_paths), l[1](X)] # g values at stopping times, l[1] 
price = 1 / test_paths * np.sum(Z) # monte carlo estimate of the price
print(f"Estimated price of the {d}D Bermudan Max Call Option: {price:.2f}")

Estimated price of the 5D Bermudan Max Call Option: 24.16
